(section-topics-real)=
# Real-valued signal challenge

Majority of real-world environments or dataset are represented using solely continue-valued inputs or alongside with discrete attributes. Unfortunately, the know ALCS implementations were not initially designed for such representations. The fundamental modification required to overcome aforementioned limitation is the adaptation of rule representation within the system. Despite the variety of possible approaches listed in this chapter each one comes with own merits and perils.

Two major concepts involving the discretization and interval encoding using dedicated alphabets will be pursued in the later section, while the hindmost ones involving the neural network and fuzzy logic will be only mentioned. There are two main reason of this decision:

1. drastic changes in rule representation requires significant modifications in existing and tightly coupled components. In some cases some of them are not usable in their proposed form - like the ALP component in ACS systems needs a complete redefinition for other alphabet encodings,
2. rule interpretability is relevantly blemished.

Therefore, even if some approaches are more appealing for the problem their usage violates major ALCS principles of creating compact set of human-readable rules or does not formally specify the behaviour of related inner mechanisms fulfilling the evolution and assessing each classifier. A more detailed research is however highly advised, but won't be pursued herein.

(section-topics-real-discretization)=
## Discretization
````{margin}
```{admonition} Ternary representation
A typical representation of a rule in LCS systems comprising three symbols - $\{0, 1, \#\}$
```
````

One of the approach for handling continuous attributes is to partition them into a number of sub-ranges and treat each sub-range as a category. This process is known as _discretization_ and is used across many families of ML algorithms in order to create better models {cite}`elomaa2004efficient`. Minding the nature of ALCS, the usage of such nominal values would be the simplest and most straightforward approach. ALCS systems by design are not limited by _ternary representation_, therefore creating arbitrary number of possible states is achievable. The first such implementation called rACS was proposed by Unold and Mianowski in 2016 {cite}`unold2016real` and tested successfully on [](section-topics-environments-corridor) and [](section-topics-environments-grid) environments. Both of them have a regular structure, meaning that the intervals are evenly distributed throughout the investigated range.

Unfortunately, the number of ways to discretize a continuous attribute is infinite. Kotsiantis in {cite}`kotsiantis2006discretization` takes a survey of possible discretization techniques, but in this work the preferred approach is to divide the search-space into $n$ equally spaced intervals, referred later as _bins_ or _buckets_. When the $n$ is large, that means that system precision is increased resulting obviously in growth of classifier population. Such a population can be further optimized by compacting the classifiers operating in the neighbouring space regions {cite}`wyatt2004building` {cite}`wilson2001compact`. On the contrary, when the $n$ is low the system under-performs due to inability of creating accurate classifiers.

The process of assigning a discrete value for each consecutive interval region is depicted on Figure {numref}`{number} <discretization-fig>`.

:::{figure-md} discretization-fig
<img src="../../_static/discretization.png" width="75%">

Representation of allele in rACS as a natural number of partition. Figure taken from {cite}`unold2016real`.
:::

Such solution for real-value representation in simple scenario does not require significant changes to any major components and retain the human-readability and interpretability of created rules. However, in situations when an arbitrary number of bins for each observation attribute is used certain modifications might be necessary (for example additional restrictions for the GA cross-over operator).

(section-topics-real-interval)=
## Interval predicates
The discretization approach assumed that a distinct value of condition attribute represents a fixed-length interval part of an input range. Another approach is to encode the signal using inequities, ie
- $x_i < \theta_i$ for problems where decision surface is a hyperplane in $n$-dimensional solution space,
- $p_i \leq x_i < q_i$ for problems where requiring hyper-rectangular solution

Such approach facilitates creation of more general and compact population size, due to arbitrarily interval ranges.

CSR, UBR, XCSR, interval predicates
XCSI

(section-topics-real-neural)=
## Neural networks
O’Hara and Bull experimented with representing the rule structure of an XCS classifier with two artificial neural networks named X-NCS {cite}`ohara2004prediction` {cite}`o2005building`. While the first one, called the _action-network_  determines the application condition of the classifiers replacing the condition-action part, the latter one - _anticipation network_ - forms the description of the predicted next state.

Both networks are fully connected multi-layered perceptrons (MLP) with the same number of nodes in their hidden layer. The input layer in both cases matches the observation state vector provided by the environment. In the action network the size of an output layer is equal to the number of possible actions incremented with an extra node signifying a non-matching situation. Hence, the anticipation network is responsible for predicting next state, the size of its output layer is equal to the input one. Figures {numref}`{number} <xncs-action-network>` and {numref}`{number} <xncs-anticipation-network>` visualize both topologies.

````{tabbed} Action Network
```{figure} ../../_static/graphs/xncs_action_network.gv.png
:name: xncs-action-network

The topology of fully connected MLP of the network determing the agent's action based on the observed state (input layer). The number of output nodes is equal to the number of possible actions with an extra state representing a non-matching case.
```
````

````{tabbed} Anticipation Network
```{figure} ../../_static/graphs/xncs_anticipation_network.gv.png
:name: xncs-anticipation-network

The topology of fully connected MLP of the network determing the anticipated states using the observed environmental state (input layer). Certain output nodes referrs to particular input nodes.
```
````

The system starts with an initial random population, containing the maximum number of classifiers considered in certain experiment, as opposed to standard XCS {cite}`wilson1995classifier`. The weights of the network are randomly initialized in the range $[-1.0, 1.0]$ and throughout the experiment are updated using two search techniques - the local search performed by backpropagation complemented by the global sampling performed by GA.

In order to assess the general system error all of internal mechanisms remained unchanged expect for the calculation of rule’s absolute error, which is defined as the sum of prediction and  lookahead error (measuring the correctness of the prediction).

The key differences between the traditional ternary representation is obviously the lack of explicit wildcard symbol and hence no explicit pass-through of input to anticipations. A concept of a reliable classifier cannot be applied in X-NCS - the anticipation accuracy is based on a percentage of accurate anticipations per presentation.

The extension was tested by authors in various configurations showing promising results especially on discrete multistep problems. Due to novel rule representation the X-NCS system is suited for real-valued data representation, but the conceptual differences makes it difficult to compare it with other systems. Aspects like vague generalization metric, constant population size or the anticipation accuracy computation would require performing dedicated research solely on this implementation.

(section-topics-real-fuzzy)=
## Fuzzy representation
Kondziela undertook an approach to create a fuzzy variant of ACS in 2021 {cite}`kondziela2021facs`. The presented idea modified the system comprising four major elements {cite}`rudnik2011koncepcja`:

- _fuzzification_ - assigning set membership of current environmental perception,
- _inference_ - aggregates the results by comparing the membership functions with collected knowledge,
- _knowledge-store_ - stores population of classifiers where representing rules using the _Fuzzy Inference System_ (Mamandi) of `IF..AND..THEN` statements {cite}`zadeh1996fuzzy`,
- _defuzzification_ - provides single value response obtained from the inference phase determining the final action value.

As the first step the vector of environment signal determines set memberships using predefined functions {cite}`casillas2007fuzzy` {cite}`bonarini2007fixcs`. Then using the rule representation described by Equation {math:numref}`fuzzy-eq` the match set is formed. Each input value $X_i$ is equal to a linguistic set of $\tilde{A_i} = \{ A_{i1} \lor \dots \lor A_{il} \}$ meaning that classifier internally consists of a rule described with $\{0, 1, \# \}$ symbols.

```{math}
:label: fuzzy-eq

\mathbf{IF}\ X_1\ \text{is}\ \tilde{A_1}\ \text{and}\ \dots \ X_n\ \text{is}\ \tilde{A_n}\ \mathbf{THEN}\ Y\ \text{is}\ B
```

In the next step the action set $[A]$ is formed in the same way as in traditional ACS, but the final action selection procedure differs - it is proposed by taking advantage of each rule's membership function values and the _Center of Gravity_ method for defuzzification {cite}`kondziela2021facs`.

Preliminary tests made on multistep, discrete maze environments showed that fuzzy ACS implementation successfully predicts more than 75% of encountered situations and maintained limited number of reliable classifiers (although both metrics were highly oscillating). Author did not report any other operational performance indicators.

The usage of fuzzy logic enables the system to naturally handle the real-valued input. The obvious impediment is the requirement to specify membership functions for each environmental perception upfront. The selection of optimal values is complicated and further increases the number of overall system tunable parameters. The other identified flaw relates to the GA phase which is not suited for new representation. Both the mutation and cross-over operators should be reviewed accordingly.